In [3]:

GYRO = True
SEQUENCE_LENGTH = 4
SEQUENCE_OVERLAP = 3
BATCH_SIZE = 10
EPOCHS = 50
MODEL_NAME = f"physical_activity_epochs:{EPOCHS}_batch:{BATCH_SIZE}_gyro:{GYRO}_window:{SEQUENCE_LENGTH}_overlap:{SEQUENCE_OVERLAP}.keras"

# EDIT THE VALUES IN THE CELL ABOVE THEN PRESS RUN ALL

In [4]:
import file_tagger
import sequence_genrator
import tensorflow as tf
import numpy as np
from keras import layers, Sequential, models
from sklearn.model_selection import train_test_split


In [5]:
DATA_DIRECTORY = "./all_respeck"
PHYSICAL_ACTIVITIES = {
    "sitting&coughing",
    "sitting&hyperventilating",
    
    "standing&coughing",
    "standing&hyperventilating",
    
    "lying_down_back&coughing",
    "lying_down_back&hyperventilating",
    
    "lying_down_stomach&coughing",
    "lying_down_stomach&hyperventilating",
    
    "lying_down_right&coughing",
    "lying_down_right&hyperventilating",
    
    "lying_down_left&coughing",
    "lying_down_left&hyperventilating",
    
    "sitting&talking",
    "sitting&eating",
    "sitting&singing",
    "sitting&laughing",
    
    "standing&talking",
    "standing&eating",
    "standing&singing",
    "standing&laughing",
    
    "lying_down_back&talking",
    "lying_down_back&singing",
    "lying_down_back&laughing",
    
    "lying_down_right&talking",
    "lying_down_right&singing",
    "lying_down_right&laughing",
    
    "lying_down_left&talking",
    "lying_down_left&singing",
    "lying_down_left&laughing",
    
    "lying_down_stomach&talking",
    "lying_down_stomach&singing",
    "lying_down_stomach&laughing",
}

In [6]:
def generate_training_data(directory, sequence_length, overlap, gyro = GYRO): # if gyro is false, only accelerometer data is used

    tagged_data = []

    # group each csv file into their respective areas
    csv_dictionary = file_tagger.tag_directory(directory)

    # iterates through each activity
    for key in PHYSICAL_ACTIVITIES:

        # iterates through each csv file for the activity 
        for csv_file in csv_dictionary[key]:
            if gyro:
                sequences = sequence_genrator.generate_sequences_from_file_with_gyroscope(directory + "/" + csv_file, sequence_length, overlap)
            else:
                sequences = sequence_genrator.generate_sequences_from_file_without_gyroscope(directory + "/" + csv_file, sequence_length, overlap)

            # iterate through each generated sequence
            for sequence in sequences:
                tagged_data.append((key, sequence))

    print ("there are " + str(len(tagged_data)) + " tagged sequences in the dataset")
    return tagged_data

In [7]:
# split data into training, dev, and test sets
def train_dev_test_split(data, labels, dev_size, test_size, random_state=42):
    # Split the data into training and temporary (dev + test) sets
    train_data, temp_data, train_labels, temp_labels = train_test_split(data, labels, test_size=(dev_size + test_size), random_state=random_state)
    
    # Split the temporary data into dev and test sets
    dev_data, test_data, dev_labels, test_labels = train_test_split(temp_data, temp_labels, 
                                                                 test_size=(test_size / (dev_size + test_size)), random_state=random_state)
    
    return train_data, dev_data, test_data, train_labels, dev_labels, test_labels

In [8]:
tagged_sequences = generate_training_data(DATA_DIRECTORY, SEQUENCE_LENGTH, SEQUENCE_OVERLAP)

# Combine all sequences and labels
sequences = [sequence for _, sequence in tagged_sequences]
labels = [label for label, _ in tagged_sequences]


# encode labels to numbers
sequences = np.array(sequences, dtype=np.float32)
label_to_index = {label: idx for idx, label in enumerate(PHYSICAL_ACTIVITIES)}
labels_encoded = [label_to_index[label] for label in labels]
labels_encoded = np.array(labels_encoded)

there are 38213 tagged sequences in the dataset


In [9]:
#CNN MODEL
def train_model_CNN(input_data, labels_encoded, unique_labels, epochs, batch_size, validation_data):
    if GYRO:
        width = 6
    else:
        width = 3
    # Define the CNN model for your specific input shape
    model = Sequential([
        layers.Conv1D(32, 3, activation='relu', input_shape=(SEQUENCE_LENGTH*25, width)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Conv1D(128, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Dropout(0.5),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(unique_labels), activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the CNN model
    model.fit(input_data, labels_encoded, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    return model

In [10]:

train_data, dev_data, test_data, train_labels, dev_labels, test_labels = train_dev_test_split(sequences, labels_encoded, dev_size=0.1, test_size=0.1) #10% dev, 10% test


# train and save model (CHOOSE BETWEEN CNN AND LSTM)
model = train_model_CNN(train_data, train_labels, PHYSICAL_ACTIVITIES, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(dev_data, dev_labels)) #batch_size, epochs


# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Save the trained model
model.save(f"models/Task2&3Test/{MODEL_NAME}_{test_accuracy}")

Epoch 1/50
3057/3057 [==============================] - 18s 5ms/step - loss: 2.1012 - accuracy: 0.1568 - val_loss: 1.9766 - val_accuracy: 0.1769
Epoch 2/50
3057/3057 [==============================] - 17s 5ms/step - loss: 1.9807 - accuracy: 0.1807 - val_loss: 1.9419 - val_accuracy: 0.1869
Epoch 3/50
3057/3057 [==============================] - 15s 5ms/step - loss: 1.7804 - accuracy: 0.2437 - val_loss: 1.5478 - val_accuracy: 0.3483
Epoch 4/50
3057/3057 [==============================] - 16s 5ms/step - loss: 1.5194 - accuracy: 0.3447 - val_loss: 1.4387 - val_accuracy: 0.3735
Epoch 5/50
3057/3057 [==============================] - 17s 6ms/step - loss: 1.4410 - accuracy: 0.3702 - val_loss: 1.3404 - val_accuracy: 0.3968
Epoch 6/50
3057/3057 [==============================] - 16s 5ms/step - loss: 1.3855 - accuracy: 0.3936 - val_loss: 1.2874 - val_accuracy: 0.4156
Epoch 7/50
3057/3057 [==============================] - 16s 5ms/step - loss: 1.3474 - accuracy: 0.4154 - val_loss: 1.2839 - val_ac

2023-11-05 03:58:05.691869: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,5,128]
	 [[{{node inputs}}]]
2023-11-05 03:58:06.223681: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,5,128]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: models/Task2&3Test/physical_activity_epochs:50_batch:10_gyro:False_window:5_overlap:4.keras_0.6386708617210388/assets


INFO:tensorflow:Assets written to: models/Task2&3Test/physical_activity_epochs:50_batch:10_gyro:False_window:5_overlap:4.keras_0.6386708617210388/assets
